In [14]:
import findspark

findspark.init()

In [15]:
import os
import sys
from pyspark.rdd import RDD
from pyspark.sql import Row
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
from pyspark.sql.functions import desc
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [16]:
def init_spark():
    spark = SparkSession \
        .builder \
        .appName("Python Spark SQL basic example") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()
    return spark

In [17]:
#spark = SparkSession.builder.appName('Recommendations').getOrCreate()
#movies = spark.read.csv("movies.csv",header=True)
spark = init_spark()
ratings = spark.read.csv("pairs_main.csv",inferSchema =True,header=True)
ratings.show()
print (ratings.dtypes)

+--------+-------+------+
|pipeline|dataset|status|
+--------+-------+------+
|       0|     10|     1|
|       1|      2|     1|
|       1|      6|     2|
|       1|      8|     1|
|       1|     10|     1|
|       1|     12|     1|
|       1|     20|     2|
|       2|      2|     1|
|       2|      5|     1|
|       2|      8|     1|
|       2|      9|     1|
|       3|      5|     2|
|       3|      6|     1|
|       3|      9|     2|
|       3|     10|     1|
|       3|     12|     1|
|       3|     13|     2|
|       3|     14|     2|
|       3|     15|     2|
|       3|     16|     1|
+--------+-------+------+
only showing top 20 rows

[('pipeline', 'int'), ('dataset', 'int'), ('status', 'int')]


In [18]:
type(ratings)

pyspark.sql.dataframe.DataFrame

In [19]:
(training, test) = ratings.randomSplit([0.8, 0.2])#, seed=2020)
test.show()

+--------+-------+------+
|pipeline|dataset|status|
+--------+-------+------+
|       1|      8|     1|
|       1|     20|     2|
|       3|      6|     1|
|       3|     10|     1|
|       3|     12|     1|
|       3|     13|     2|
|       3|     14|     2|
|       4|      6|     1|
|       4|     11|     1|
|       4|     18|     1|
|       4|     20|     2|
|       6|     14|     2|
|       7|     14|     2|
|       8|     12|     1|
|       9|     10|     1|
|      10|      8|     1|
|      11|     10|     1|
|      12|      3|     2|
|      12|      9|     2|
|      12|     16|     2|
+--------+-------+------+
only showing top 20 rows



In [20]:
print(type(training))

<class 'pyspark.sql.dataframe.DataFrame'>


In [21]:
als = ALS(maxIter=5, regParam=0.01, userCol="pipeline", itemCol="dataset",ratingCol="status", coldStartStrategy="drop", rank=10)#, seed=2020)

model = als.fit(training)

predictions = model.transform(test)




In [10]:
print(type(predictions))

<class 'pyspark.sql.dataframe.DataFrame'>


In [11]:
predictions.show()

+--------+-------+------+----------+
|pipeline|dataset|status|prediction|
+--------+-------+------+----------+
|      12|     12|     1| 0.8723879|
|      15|     12|     2| 0.9684528|
|      25|     12|     2| 0.9476479|
|      14|     12|     1|0.92244697|
|      28|      1|     1| 0.8915687|
|      16|     13|     2| 2.0215654|
|       4|     13|     1| 1.8872836|
|      24|     13|     2| 2.0633154|
|      28|      6|     2|  0.597535|
|      20|      6|     1|  0.923225|
|       8|      6|     1|  1.068658|
|       7|      6|     2| 1.2698269|
|       6|     16|     1|  1.909276|
|       4|     16|     2| 1.6329396|
|      12|      3|     2| 1.6681443|
|      28|     20|     2| 1.8239863|
|      12|     20|     2| 1.7991782|
|      13|     20|     2| 1.8685272|
|       6|     20|     2| 1.8617117|
|       3|     20|     1| 1.2666792|
+--------+-------+------+----------+
only showing top 20 rows



In [1]:
print(predictions.loc[:,'prediction'])

NameError: name 'predictions' is not defined

In [ ]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="status", predictionCol="prediction")
evaluator.evaluate(predictions)